# Explorative  Analysis

## Import packages

In [ ]:
import pandas as pd
import numpy as np
import ast
from datetime import datetime
import matplotlib.pyplot as plt
import matplotlib.ticker as tkr
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.preprocessing import MinMaxScaler

pd.set_option('display.max_columns', None)


## Plot parameters

In [ ]:
sns.set_palette(['silver', 'skyblue', 'darkorange'], n_colors=3)
sns.despine(right=False, top=False)

plt.rcParams.update({'font.size': 18,
                     'axes.edgecolor': 'gray',
                     'axes.labelcolor': 'gray',
                     'axes.labelweight': 'ultralight',
                     'xtick.color': 'gray',
                     'xtick.labelsize': 'small',
                     'ytick.color': 'gray',
                     'ytick.labelsize': 'small',
                     'axes.titlelocation': 'center',
                     'yaxis.labellocation': 'top',
                     'patch.edgecolor': 'gray',
                     'axes.titlecolor': 'dimgray',
                     'axes.formatter.useoffset': False,
                     'axes.formatter.use_mathtext': True,
                     'figure.facecolor': 'white',
                     'axes.facecolor': 'white',
                     'savefig.facecolor': 'white',
                     "axes.labelsize": 18,
                     'figure.figsize': [8, 8]
                     }
                    )

# plt.rcParams.keys()


## Read data
- The dataset has 21983 rows and 30 columns
- Every row of the dataset represents one customer who has registered to use the service during September 2019 and either has or hasn’t made orders during the time from then and October 2020.

In [ ]:
# Path of the csv file containing wolt user data
user_data_path = "data\dataset_for_analyst_assignment_20201120.csv"

# Read provided data from csv file
user_data_df = pd.read_csv(user_data_path)


In [ ]:
# Check dataset size
user_data_df.shape


In [ ]:
# Check first 5 rows of teh raw data
user_data_df.head()


## Data wrangling

In [ ]:
# Change data type to date for columns representing a date
user_data_df[["REGISTRATION_DATE", "FIRST_PURCHASE_DAY", "LAST_PURCHASE_DAY"]] = user_data_df[[
    "REGISTRATION_DATE", "FIRST_PURCHASE_DAY", "LAST_PURCHASE_DAY"]].apply(pd.to_datetime)

# Check column data types
user_data_df.dtypes


### Explode column PURCHASE_COUNT_BY_STORE_TYPE

In [ ]:
# Data frame with columns purchase count for each store type
store_type_df = (user_data_df["PURCHASE_COUNT_BY_STORE_TYPE"].replace('\n', '', regex=True)
                 .apply(ast.literal_eval)
                 .apply(pd.Series)
                 .rename(columns={"General merchandise": "PURCHASE_COUNT_GENERAL_MERCHANDISE",
                                  "Grocery": "PURCHASE_COUNT_GROCERY",
                                  "Pet supplies": "PURCHASE_COUNT_PET_SUPPLIES",
                                  "Restaurant": "PURCHASE_COUNT_RESTAURANT",
                                  "Retail store": "PURCHASE_COUNT_RETAIL_STORE"})
                 )

# Remove old column for PURCHASE_COUNT_BY_STORE_TYPE and add the individual columns with purchase count for each store type
user_data_df = pd.concat([user_data_df.drop(
    ["PURCHASE_COUNT_BY_STORE_TYPE"], axis=1), store_type_df], axis=1)


### Check data quality


#### Missing values
- The columns REGISTRATION_COUNTRY, PURCHASE_COUNT, PURCHASE_COUNT_BY_STORE_TYPE, USER_ID, USER_HAS_VALID_PAYMENT_METHOD, REGISTRATION_DATE have no missing data
- Most columns for users that made no purchase are expected to be empty
- PREFERRED_RESTAURANT_TYPES has 87.7% missing values

In [ ]:
# Sum all missing values for each column
total = user_data_df.isnull().sum().sort_values(ascending=False)

# Calculate the percentage of missing values for each column
percent = round(user_data_df.isnull().sum(
)/user_data_df.isnull().count()*100, 1).sort_values(ascending=False)

# Create a data frame containing the total number of missing values and the % out of the total number of values
missing_data = pd.concat([total, percent], axis=1, keys=['Total', '%'])

# Check the % of missing values for each column
missing_data


##### Missing values for users that made at least one purchase
- PREFERRED_RESTAURANT_TYPES has a high share of missing values even for users that made at least one purchase

In [ ]:
# Sum all missing values for each column
total = user_data_df.query("PURCHASE_COUNT > 0").isnull(
).sum().sort_values(ascending=False)

# Calculate the percentage of missing values for each column
percent = round(user_data_df.query("PURCHASE_COUNT > 0").isnull().sum(
)/user_data_df.query("PURCHASE_COUNT > 0").isnull().count()*100, 1).sort_values(ascending=False)

# Create a data frame containing the total number of missing values and the % out of the total number of values
missing_data = pd.concat([total, percent], axis=1, keys=['Total', '%'])

# Check the % of missing values for each column
missing_data.query("Total > 0")


#### Is registration date during September 2019?
- Yes all registration dates are during 2019-09

In [ ]:
# Check if all registration dates are from 2019-09
registration_date_s = user_data_df["REGISTRATION_DATE"].dt.to_period('M')
set(registration_date_s)


#### Are all user_ids unique?
- Yes all user_ids are unique

In [ ]:
# Users IDs count and distinct count to check for duplicates
user_data_df.agg(count_users=("USER_ID", "count"),
                 distinct=("USER_ID", "nunique"))


#### Are there any negative purchase_count?
- no negative values as expected

In [ ]:
# Count negative purchases
user_data_df.query("PURCHASE_COUNT < 0")["PURCHASE_COUNT"].count()


#### Is purchase_count = ios_purchase + web_purchases + android_purchases?
- Yes

In [ ]:
# Check if ios_purchase + web_purchases + android_purchases
user_data_df.query(
    "PURCHASE_COUNT != IOS_PURCHASES + WEB_PURCHASES + ANDROID_PURCHASES and PURCHASE_COUNT > 0")["PURCHASE_COUNT"].count()


#### Is purchase_count = purchase_count_delivery + purchase_count_takeaway?
- Yes

In [ ]:
# purchase_count_delivery + purchase_count_takeaway
user_data_df.query(
    "PURCHASE_COUNT != PURCHASE_COUNT_DELIVERY + PURCHASE_COUNT_TAKEAWAY and PURCHASE_COUNT > 0")["PURCHASE_COUNT"].count()


### New column preferred_device_corr 
- For users that made no purchase the data is the same and represents the device the user used to register, but for users that made at least one purchase this column represents the device with most purchases

In [ ]:
# Create column PREFERRED_DEVICE_CORR
user_data_df["PREFERRED_DEVICE_CORR"] = np.where((user_data_df["IOS_PURCHASES"] > user_data_df["WEB_PURCHASES"])
                                                 & (user_data_df["IOS_PURCHASES"] > user_data_df["ANDROID_PURCHASES"]), "ios",
                                                 np.where((user_data_df["ANDROID_PURCHASES"] > user_data_df["WEB_PURCHASES"])
                                                          & (user_data_df["ANDROID_PURCHASES"] > user_data_df["IOS_PURCHASES"]), "android",
                                                          np.where((user_data_df["WEB_PURCHASES"] > user_data_df["ANDROID_PURCHASES"])
                                                                   & (user_data_df["WEB_PURCHASES"] > user_data_df["IOS_PURCHASES"]), "web", user_data_df["PREFERRED_DEVICE"]
                                                                   )))


### New column for days from last purchase
- I am assuming that this analysis would have been used some time after October 2020 to reactivate users that registered in September 2019. So this column was calculated as the difference in days between last purchase day and 2020-10-31.

In [ ]:
# Calculate column for days from last purchase
user_data_df["DAYS_FROM_LAST_PURCHASE"] = (
    datetime(2020, 10, 31) - user_data_df["LAST_PURCHASE_DAY"]).dt.days

# Calculate column for days to first purchase
user_data_df["DAYS_TO_FIRST_PURCHASE"] = (
    user_data_df["FIRST_PURCHASE_DAY"] - user_data_df["REGISTRATION_DATE"]).dt.days


### Column for purchase count segments

In [ ]:
# Column for purchase count segments
user_data_df["PURCHASE_COUNT_SEGM"] = np.where(user_data_df["PURCHASE_COUNT"] == 0, "inactive",
                                               np.where(user_data_df["PURCHASE_COUNT"] == 1, "one_time_shoppers",
                                                        np.where((user_data_df["PURCHASE_COUNT"] >= 2) & (user_data_df["PURCHASE_COUNT"] <= 20), "casual_shoppers", "frequent_shoppers"
                                                                 )))


### Column for days from last purchase segments

In [ ]:
# Column for days from last purchase segments
user_data_df["DAYS_FROM_LAST_PURCHASE_segm"] = np.where(
    user_data_df["DAYS_FROM_LAST_PURCHASE"] <= 90, "recent", "old")


## NEW DATA FRAMES

In [ ]:
# Select data only for top 3 countries accounting for 97% of the data
# Remove users that have no data for preferred device
# Remove irrelevant columns
user_data_filtered_df = (user_data_df[(user_data_df["REGISTRATION_COUNTRY"].isin(['FIN', 'DNK', 'GRC']))
                                      & (~user_data_df["PREFERRED_DEVICE_CORR"].isnull())
                                      ]
                         .reset_index()
                         .drop(["index", "PREFERRED_RESTAURANT_TYPES", "PREFERRED_DEVICE", "LATE_NIGHT_PURCHASES"], axis=1)
                         )


# Rename values for payment method column
user_data_filtered_df["USER_HAS_VALID_PAYMENT_METHOD"] = np.where(
    user_data_filtered_df["USER_HAS_VALID_PAYMENT_METHOD"] == True, "valid", "invalid")


In [ ]:
# Data frame for users that made at least one purchase
active_users_df = user_data_filtered_df.query(
    "PURCHASE_COUNT > 0").reset_index().drop(columns=["index"], axis=1)


### New column for purchase count groups

In [ ]:
# Column for purchase count groups
active_users_df["PURCHASE_COUNT_BIN"] = (
    np.where((active_users_df["PURCHASE_COUNT"] == 1), "1",
             np.where((active_users_df["PURCHASE_COUNT"] > 1) & (active_users_df["PURCHASE_COUNT"] <= 5), "2-5",
                      np.where((active_users_df["PURCHASE_COUNT"] > 5) & (active_users_df["PURCHASE_COUNT"] <= 10), "6-10",
                               np.where((active_users_df["PURCHASE_COUNT"] > 10) & (active_users_df["PURCHASE_COUNT"] <= 20), "11-20",
                                        np.where((active_users_df["PURCHASE_COUNT"] > 20) & (active_users_df["PURCHASE_COUNT"] <= 50), "21-50",
                                                 ">50"
                                                 )))))
)


## Data visualization

### How many users made no purchase in 1 year after registering?

In [ ]:
# Add column to check if a users made at least one purchase
user_data_filtered_df["USER_HAS_MADE_MIN_ONE_PURCHASE"] = np.where(
    user_data_filtered_df["PURCHASE_COUNT"] > 0, "Yes", "No")


In [ ]:
# Count unique users & percentage users per user type
counts = user_data_filtered_df["USER_HAS_MADE_MIN_ONE_PURCHASE"].value_counts()
percent = user_data_filtered_df["USER_HAS_MADE_MIN_ONE_PURCHASE"].value_counts(normalize=True)
user_per_df = pd.DataFrame({'Users': counts, 'Percentage': percent})

# Percentage of users that have made at least one purchase or not
user_per_df.style.format({'Percentage': "{:.2%}"})


### No of users per country
- FIN, DNK & GRC account for 97% of the data

In [ ]:
# Unique countries
user_data_df["REGISTRATION_COUNTRY"].unique().shape

In [ ]:
# Count unique users & percentage users per registration country
counts = user_data_df["REGISTRATION_COUNTRY"].value_counts()
percent = user_data_df["REGISTRATION_COUNTRY"].value_counts(normalize=True)
country_per_df = pd.DataFrame({'Users': counts, 'Percentage': percent}).reset_index()

# Percentage of users per registration country
country_per_df.head(10).style.format({'Percentage': "{:.2%}"})

In [ ]:
# Plot number of users per registration country
g = sns.countplot(data=user_data_filtered_df,
                  x="REGISTRATION_COUNTRY",
                  palette=sns.color_palette(['silver']),
                  order=['FIN', 'DNK', 'GRC']
                  )

g.set(xlabel="", ylabel="# USERS")
g.set_title("Number of registered users for top 3 countries",
            fontsize=20, loc="left", pad=14)
sns.despine()


In [ ]:
# Plot number of active and inactive users per registration country
g = sns.countplot(data=user_data_filtered_df,
                  x="REGISTRATION_COUNTRY",
                  palette=sns.color_palette(['#00c4e1', '#980000']),
                  order=['FIN', 'DNK', 'GRC'],
                  hue="USER_HAS_MADE_MIN_ONE_PURCHASE",
                  hue_order=["Yes", "No"]
                  )

g.set(xlabel="", ylabel="# USERS")
g.set_title("Number of active and inactive users",
            fontsize=20, loc="left", pad=14)
g.get_legend().remove()
sns.despine()


### No of users per prefered device
- On total most users prefer ios

In [ ]:
# Unique preferred device corrected column
user_data_filtered_df["PREFERRED_DEVICE_CORR"].unique()

In [ ]:
# Data frame with percentage of users per preferred device
counts = user_data_filtered_df["PREFERRED_DEVICE_CORR"].value_counts()
percent = user_data_filtered_df["PREFERRED_DEVICE_CORR"].value_counts(normalize=True)
device_per_df = pd.DataFrame({'Users': counts, 'Percentage': percent}).reset_index()

# Percentage of users per preferred device
device_per_df.head(10).style.format({'Percentage': "{:.2%}"})

In [ ]:
# Plot number of user per device
g = sns.countplot(data=user_data_filtered_df.query("REGISTRATION_COUNTRY == 'GRC' and PURCHASE_COUNT_SEGM == 'inactive'"),
                  x="PREFERRED_DEVICE_CORR",
                  palette=sns.color_palette(['silver']),
                  order=['ios', 'android', 'web']
                  )

g.set(xlabel="", ylabel="# USERS")
g.set_title("Number of users by preferred device",
            fontsize=18, loc="left", pad=14)
sns.despine()


In [ ]:
# Plot number of users per registration country and device
g = sns.catplot(data=user_data_filtered_df,
                x="PREFERRED_DEVICE_CORR",
                col="REGISTRATION_COUNTRY",
                kind="count",
                order=['ios', 'android', 'web'],
                col_order=["FIN", "DNK", "GRC"],
                palette=sns.color_palette(['silver'])
                )

g.set_ylabels("# USERS", fontsize=18)
g.set_xlabels("")
g.set_titles("{col_name}")


### No of users per valid payment method

In [ ]:
# Number of users per registration country and payment validity
user_data_filtered_df.groupby(["REGISTRATION_COUNTRY", "USER_HAS_VALID_PAYMENT_METHOD"]).agg(
    users=("USER_ID", "nunique"))


In [ ]:
# Plot number of users per registration country and payment validity
g = sns.countplot(data=user_data_filtered_df,
                  x="REGISTRATION_COUNTRY",
                  palette=sns.color_palette(['#00c4e1', '#980000']),
                  order=['FIN', 'DNK', 'GRC'],
                  hue="USER_HAS_VALID_PAYMENT_METHOD",
                  hue_order=["valid", "invalid"]
                  )

g.set(xlabel="", ylabel="# USERS")
g.set_title("Number of users by payment method",
            fontsize=20, loc="left", pad=14)
g.get_legend().remove()
sns.despine()


### No of active and inactive users by country and device

In [ ]:
# Plot active and inactive users by registration country and device
g = sns.catplot(data=user_data_filtered_df,
                x="PREFERRED_DEVICE_CORR",
                col="REGISTRATION_COUNTRY",
                hue="USER_HAS_MADE_MIN_ONE_PURCHASE",
                hue_order=["Yes", "No"],
                kind="count",
                order=['ios', 'android', 'web'],
                col_order=["FIN", "DNK", "GRC"],
                palette=sns.color_palette(['#00c4e1', '#980000']),
                legend=False
                )

g.set_ylabels("# USERS", fontsize=18)
g.set_xlabels("")
g.set_titles("{col_name}")


### No of active and inactive users by country and payment validity

In [ ]:
# Plot active and inactive users by registration country and paymeny validity
g = sns.catplot(data=user_data_filtered_df,
                x="USER_HAS_VALID_PAYMENT_METHOD",
                col="REGISTRATION_COUNTRY",
                hue="USER_HAS_MADE_MIN_ONE_PURCHASE",
                hue_order=["Yes", "No"],
                kind="count",
                order=["valid", "invalid"],
                col_order=["FIN", "DNK", "GRC"],
                palette=sns.color_palette(['#00c4e1', '#980000']),
                legend=False
                )

g.set_ylabels("# USERS", fontsize=18)
g.set_xlabels("")
g.set_titles("{col_name}")


### Distribution of purchase count for active users

In [ ]:
# Plot number of active users by registration country and purchase groups
g = sns.catplot(data=active_users_df,
                x="PURCHASE_COUNT_BIN",
                col="REGISTRATION_COUNTRY",
                kind="count",
                order=["1", "2-5", "6-10", "11-20", "21-50", ">50"],
                col_order=["FIN", "DNK", "GRC"],
                palette=sns.color_palette(
                    ['#00c4e1', '#00c4e1', '#C0C0C0', '#C0C0C0', '#C0C0C0', '#980000']),
                legend=False,
                col_wrap=3,
                aspect=1,
                height=6
                )

g.set_ylabels("# ACTIVE USERS", fontsize=18)
g.set_xlabels("")
g.set_titles("{col_name}")


In [ ]:
# Plot total purchase value in eur by registration country and purchase groups
g = sns.catplot(data=active_users_df.groupby(["REGISTRATION_COUNTRY", "PURCHASE_COUNT_BIN"]).agg(sum_purchases_eur=("TOTAL_PURCHASES_EUR", "sum")).reset_index(),
                x="PURCHASE_COUNT_BIN",
                y="sum_purchases_eur",
                col="REGISTRATION_COUNTRY",
                kind="bar",
                order=["1", "2-5", "6-10", "11-20", "21-50", ">50"],
                col_order=["FIN", "DNK", "GRC"],
                palette=sns.color_palette(
                    ['#C0C0C0', '#C0C0C0', '#C0C0C0', '#00c4e1', '#00c4e1', '#C0C0C0']),
                legend=False,
                col_wrap=3,
                aspect=1,
                height=6
                )

g.set_ylabels("TOTAL PURCHASES EUR", fontsize=18)
g.set_xlabels("")
g.set_titles("{col_name}")

# Format y tick labels
for ax in g.axes.flat:
    ax.yaxis.set_major_formatter(
        tkr.FuncFormatter(lambda y, p: f'{y/1000:.0f}K €'))


### Distribution of avg purchase value in EUR

In [ ]:
# Median and mean for avg purchase value in eur
active_users_df.groupby(["REGISTRATION_COUNTRY"]).agg(mean=(
    "AVG_PURCHASE_VALUE_EUR", "mean"), median=("AVG_PURCHASE_VALUE_EUR", "median"))


In [ ]:
# Plot distribution of avg purchase value per registration country
g = sns.catplot(data=active_users_df.query("AVG_PURCHASE_VALUE_EUR <= 150"),
                x="REGISTRATION_COUNTRY",
                y="AVG_PURCHASE_VALUE_EUR",
                kind="boxen",
                order=["FIN", "DNK", "GRC"],
                palette=sns.color_palette(['silver']),
                legend=False,
                height=7
                )

g.set_ylabels("AVG PURCHASE VALUE EUR*", fontsize=18)
g.set_xlabels("")
g.set_titles("{col_name}")

# Format y tick labels
for ax in g.axes.flat:
    ax.yaxis.set_major_formatter(tkr.FuncFormatter(lambda y, p: f'{y:.0f} €'))


### Distribution of delivery purchases vs takeaway purchases for active users

In [ ]:
# Dataframe to compare delivery vs takeaway
delivery_vs_takeaway_df = pd.melt(active_users_df[["USER_ID", "REGISTRATION_COUNTRY", "PURCHASE_COUNT_DELIVERY", "PURCHASE_COUNT_TAKEAWAY"]]
                                  .rename(columns={"PURCHASE_COUNT_DELIVERY": "delivery",
                                                   "PURCHASE_COUNT_TAKEAWAY": "takeaway"}),
                                  id_vars=["USER_ID", "REGISTRATION_COUNTRY"], value_vars=["delivery", "takeaway"], var_name="PURCHASE_TYPE", value_name="PURCHASE_COUNT"
                                  )


In [ ]:
# Create columns to differentiate delivery only users and takeaway only users
active_users_df["USERS_DELIVERY_AND_TAKEAWAY"] = np.where((active_users_df["PURCHASE_COUNT_DELIVERY"] > 0) & (active_users_df["PURCHASE_COUNT_TAKEAWAY"] > 0), 1, 0)
active_users_df["USERS_DELIVERY_ONLY"] = np.where((active_users_df["PURCHASE_COUNT_DELIVERY"] > 0) & (active_users_df["PURCHASE_COUNT_TAKEAWAY"] == 0), 1, 0)
active_users_df["USERS_TAKEAWAY_ONLY"] = np.where((active_users_df["PURCHASE_COUNT_DELIVERY"] == 0) & (active_users_df["PURCHASE_COUNT_TAKEAWAY"] > 0), 1, 0)

In [ ]:
# Dataframe to compare delivery vs takeaway
delivery_vs_takeaway_df_2 = (active_users_df.groupby(["REGISTRATION_COUNTRY"])
                             .agg(purchase_count_total=("PURCHASE_COUNT", "sum"),
                                  users_total=("USER_ID", "nunique"),
                                  users_delivery_and_takeaway=(
                                      "USERS_DELIVERY_AND_TAKEAWAY", "sum"),
                                  purchase_count_delivery=(
                                      "PURCHASE_COUNT_DELIVERY", "sum"),
                                  users_delivery_only=(
                                      "USERS_DELIVERY_ONLY", "sum"),
                                  purchase_count_takeaway=(
                                      "PURCHASE_COUNT_TAKEAWAY", "sum"),
                                  users_takeaway_only=(
                                      "USERS_TAKEAWAY_ONLY", "sum")
                                  )
                             .reset_index()
                             )


delivery_vs_takeaway_df_2["purchase_count_delivery_per"] = delivery_vs_takeaway_df_2["purchase_count_delivery"] / \
    delivery_vs_takeaway_df_2["purchase_count_total"]
delivery_vs_takeaway_df_2["purchase_count_takeaway_per"] = delivery_vs_takeaway_df_2["purchase_count_takeaway"] / \
    delivery_vs_takeaway_df_2["purchase_count_total"]


In [ ]:
# Plot share of delivery vs takeaway purchases by registration country
plot_df = (delivery_vs_takeaway_df_2[["REGISTRATION_COUNTRY", "purchase_count_delivery_per", "purchase_count_takeaway_per"]]
           .melt(id_vars=['REGISTRATION_COUNTRY'], var_name='purchase_type', value_name='purchase_count_per')
           .reset_index()
           )

g = sns.catplot(data=plot_df,
                x="REGISTRATION_COUNTRY",
                y="purchase_count_per",
                hue="purchase_type",
                hue_order=["purchase_count_delivery_per", "purchase_count_takeaway_per"],
                kind="bar",
                order=["FIN", "DNK", "GRC"],
                palette=sns.color_palette(['#00c4e1', '#980000']),
                legend=False, 
                height=6
                )

g.set_ylabels("COUNT PURCHASES %", fontsize=18)
g.set_xlabels("")

# Format y tick labels
for ax in g.axes.flat:
    ax.yaxis.set_major_formatter(tkr.FuncFormatter(lambda y, p: f'{y*100:.0f}%'))


In [ ]:
# Plot delivery only users, takeaway only users and deliver and takeaway users by registration country
plot_df = (delivery_vs_takeaway_df_2[["REGISTRATION_COUNTRY", "users_delivery_and_takeaway", "users_delivery_only", "users_takeaway_only"]]
           .melt(id_vars=['REGISTRATION_COUNTRY'], var_name='user_type', value_name='no_users')
           )

g = sns.catplot(data=plot_df,
                x="REGISTRATION_COUNTRY",
                y="no_users",
                hue="user_type",
                hue_order=["users_delivery_and_takeaway",
                           "users_delivery_only", "users_takeaway_only"],
                kind="bar",
                order=["FIN", "DNK", "GRC"],
                palette=sns.color_palette(['#ff9900', '#00c4e1', '#980000']),
                legend=False,
                height=6,
                aspect=1.2
                )

g.set_ylabels("# ACTIVE USERS", fontsize=18)
g.set_xlabels("")


### Distribution of purchases by meal type

In [ ]:
# Data frame to compare purchases for breakfast, lunch, dinner and evening
meal_type_df = (active_users_df.groupby(["REGISTRATION_COUNTRY"])
                .agg(Breakfast=("BREAKFAST_PURCHASES", "sum"),
                     Lunch=("LUNCH_PURCHASES", "sum"),
                     Dinner=("DINNER_PURCHASES", "sum"),
                     Evening=("EVENING_PURCHASES", "sum"),
                     purchases_total=("PURCHASE_COUNT", "sum")
                     )
                .reset_index()
                )
meal_type_df[["Breakfast", "Lunch", "Dinner", "Evening", "purchases_total"]] = meal_type_df[[
    "Breakfast", "Lunch", "Dinner", "Evening", "purchases_total"]].div(meal_type_df["purchases_total"], axis=0)


In [ ]:
meal_type_df

In [ ]:
# Plot share of purchase for brakfast, lunch, dinner and evening
plot_df = (meal_type_df.drop("purchases_total", axis=1)
           .melt(id_vars=['REGISTRATION_COUNTRY'], var_name='meal_type', value_name='purchase_count_per')
           .reset_index()
           )

g = sns.catplot(data=plot_df,
                x="purchase_count_per",
                y="meal_type",
                col="REGISTRATION_COUNTRY",
                kind="bar",
                order=["Breakfast", "Lunch", "Dinner", "Evening"],
                col_order=["FIN", "DNK", "GRC"],
                palette=sns.color_palette(
                    ['#C0C0C0', '#00c4e1', '#00c4e1', '#C0C0C0']),
                legend=False,
                height=6
                )

g.set_ylabels("", fontsize=18)
g.set_xlabels("COUNT PURCHASES %")
g.set_titles("{col_name}")

# Format y tick labels
for ax in g.axes.flat:
    ax.xaxis.set_major_formatter(
        tkr.FuncFormatter(lambda x, p: f'{x*100:.0f}%'))


### Distribution of users by most common hour to purchase
- This column does not make sense with an even distribution

In [ ]:
# Data frame aggregated on country and most common hour to purchase
time_of_day_df = (active_users_df.groupby(["REGISTRATION_COUNTRY", "MOST_COMMON_HOUR_OF_THE_DAY_TO_PURCHASE"])
                  .agg(users=("USER_ID", "nunique"),
                       purchases_total=("PURCHASE_COUNT", "sum")
                       )
                  .reset_index()
                  )


In [ ]:
# Plot purchases by registration country and most common hour of day to purchase
g = sns.catplot(data=time_of_day_df,
                x="MOST_COMMON_HOUR_OF_THE_DAY_TO_PURCHASE",
                y="purchases_total",
                col="REGISTRATION_COUNTRY",
                kind="bar",
                col_order=["FIN", "DNK", "GRC"],
                palette=sns.color_palette(['#C0C0C0']),
                legend=False,
                height=6.5,
                aspect=1.3
                )

g.set_ylabels("PURCHASE COUNT", fontsize=18)
g.set_xlabels("")
g.set_titles("{col_name}")

# Format y tick labels
for ax in g.axes.flat:
    ax.xaxis.set_major_formatter(tkr.FuncFormatter(lambda x, p: f'{x:.0f}'))


### Distribution of purchase venue count
- most users buy from < 5 venues

In [ ]:
# Distribution of purchase venue count
g = sns.histplot(data=active_users_df.query("DISTINCT_PURCHASE_VENUE_COUNT <= 40"),
                 x="DISTINCT_PURCHASE_VENUE_COUNT",
                 stat="count",
                 palette=sns.color_palette(['silver']),
                 discrete=True
                 )

g.set(xlabel="PURCHASE VENUE COUNT*", ylabel="# ACTIVE USERS")
sns.despine()


### Distribution of purchases by weekday
- no day sticks out

In [ ]:
# Plot number of users by most common weekday to purchase
plot_df = (active_users_df.groupby(["REGISTRATION_COUNTRY", "MOST_COMMON_WEEKDAY_TO_PURCHASE"])
           .agg(users=("USER_ID", "nunique"),
                purchases_total=("PURCHASE_COUNT", "sum")
                )
           .reset_index()
           )

g = sns.catplot(data=plot_df,
                x="MOST_COMMON_WEEKDAY_TO_PURCHASE",
                y="users",
                col="REGISTRATION_COUNTRY",
                kind="bar",
                col_order=["FIN", "DNK", "GRC"],
                palette=sns.color_palette(['#C0C0C0']),
                legend=False,
                height=6.5,
                aspect=1
                )

g.set_ylabels("# ACTIVE USERS", fontsize=18)
g.set_xlabels("")
g.set_titles("{col_name}")

# Format y tick labels
for ax in g.axes.flat:
    ax.xaxis.set_major_formatter(tkr.FuncFormatter(lambda x, p: f'{x:.0f}'))


### Distribution of purchases by store type

In [ ]:
# Data frame to compare purchases for diffrent store types
store_type_df = (active_users_df
                 .groupby(["REGISTRATION_COUNTRY"])
                 .agg(general_merchandise=("PURCHASE_COUNT_GENERAL_MERCHANDISE", "sum"),
                      Grocery=("PURCHASE_COUNT_GROCERY", "sum"),
                      pet_supplies=("PURCHASE_COUNT_PET_SUPPLIES", "sum"),
                      Restaurant=("PURCHASE_COUNT_RESTAURANT", "sum"),
                      retail_store=("PURCHASE_COUNT_RETAIL_STORE", "sum"),
                      purchases_total=("PURCHASE_COUNT", "sum")
                      )
                 .reset_index()
                 .rename(columns={"general_merchandise": "General merchandise",
                                  "pet_supplies": "Pet supplies",
                                  "retail_store": "Retail store"
                                  })
                 )
store_type_df[["General merchandise", "Grocery", "Pet supplies", "Restaurant", "Retail store"]] = store_type_df[[
    "General merchandise", "Grocery", "Pet supplies", "Restaurant", "Retail store"]].div(store_type_df["purchases_total"], axis=0)


In [ ]:
# Plot share of purchases by store type
plot_df = (store_type_df.drop("purchases_total", axis=1)
           .melt(id_vars=['REGISTRATION_COUNTRY'], var_name='store_type', value_name='purchase_count_per')
           .reset_index()
           )

g = sns.catplot(data=plot_df,
                x="purchase_count_per",
                y="store_type",
                col="REGISTRATION_COUNTRY",
                kind="bar",
                order=["Restaurant", "Retail store", "Grocery",
                       "General merchandise", "Pet supplies"],
                col_order=["FIN", "DNK", "GRC"],
                palette=sns.color_palette(
                    ['#00c4e1', '#C0C0C0', '#C0C0C0', '#C0C0C0', '#C0C0C0']),
                legend=False,
                height=6
                )

g.set_ylabels("", fontsize=18)
g.set_xlabels("COUNT PURCHASES %")
g.set_titles("{col_name}")

# Format y tick labels
for ax in g.axes.flat:
    ax.xaxis.set_major_formatter(
        tkr.FuncFormatter(lambda x, p: f'{x*100:.0f}%'))


### Distribution avg days between purchases

In [ ]:
# Plot distribution of avg days between purchases by purchase groups and country
g = sns.catplot(data=active_users_df,
                x="PURCHASE_COUNT_BIN",
                y="AVG_DAYS_BETWEEN_PURCHASES",
                col="REGISTRATION_COUNTRY",
                kind="boxen",
                order=["1", "2-5", "6-10", "11-20", "21-50", ">50"],
                col_order=["FIN", "DNK", "GRC"],
                palette=sns.color_palette(['silver']),
                legend=False,
                height=7
                )

g.set_ylabels("AVG DAYS BETWEEN PURCHASES", fontsize=18)
g.set_xlabels("")
g.set_titles("{col_name}")


### Distribution median days between purchases

In [ ]:
# Plot distribution of median days between purchases by purchase groups and country
g = sns.catplot(data=active_users_df,
                x="PURCHASE_COUNT_BIN",
                y="MEDIAN_DAYS_BETWEEN_PURCHASES",
                col="REGISTRATION_COUNTRY",
                kind="boxen",
                order=["1", "2-5", "6-10", "11-20", "21-50", ">50"],
                col_order=["FIN", "DNK", "GRC"],
                palette=sns.color_palette(['silver']),
                legend=False,
                height=7
                )

g.set_ylabels("MEDIAN DAYS BETWEEN PURCHASES", fontsize=18)
g.set_xlabels("")
g.set_titles("{col_name}")


### Distribution avg delivery distance

In [ ]:
# Column for delivery distance
active_users_df["AVERAGE_DELIVERY_DISTANCE_KMS_BINS"] = (
    np.where((active_users_df["AVERAGE_DELIVERY_DISTANCE_KMS"] < 1), "<1KM",
             np.where((active_users_df["AVERAGE_DELIVERY_DISTANCE_KMS"] >= 1) & (active_users_df["AVERAGE_DELIVERY_DISTANCE_KMS"] < 2), "1-2KM",
                      np.where((active_users_df["AVERAGE_DELIVERY_DISTANCE_KMS"] >= 2) & (active_users_df["AVERAGE_DELIVERY_DISTANCE_KMS"] < 5), "2-5KM",
                               np.where((active_users_df["AVERAGE_DELIVERY_DISTANCE_KMS"] >= 5) & (active_users_df["AVERAGE_DELIVERY_DISTANCE_KMS"] < 10), "5-10KM", ">=10KM"
                                                 ))))
)

# Dataframe to check delivery distance distribution
distance_df = (active_users_df.groupby(["REGISTRATION_COUNTRY", "AVERAGE_DELIVERY_DISTANCE_KMS_BINS"])
               .agg(users=("USER_ID", "nunique"),
                    purchases_total=("PURCHASE_COUNT", "sum")
                    )
               .reset_index()
               )

In [ ]:
# Plot number of user by delivery distance groups and country
g = sns.catplot(data=distance_df,
                x="AVERAGE_DELIVERY_DISTANCE_KMS_BINS",
                y="users",
                col="REGISTRATION_COUNTRY",
                kind="bar",
                order=["<1KM", "1-2KM", "2-5KM", "5-10KM", ">=10KM"],
                col_order=["FIN", "DNK", "GRC"],
                palette=sns.color_palette(
                    ['#C0C0C0', '#C0C0C0', '#C0C0C0', '#00c4e1', '#C0C0C0', '#C0C0C0']),
                legend=False,
                height=6.5,
                aspect=1
                )

g.set_ylabels("# ACTIVE USERS", fontsize=18)
g.set_xlabels("")
g.set_titles("{col_name}")


### Distribution days from last purchase

In [ ]:
# Plot distribution days from last purchase by country and purchase groups
g = sns.catplot(data=active_users_df,
                x="PURCHASE_COUNT_BIN",
                y="DAYS_FROM_LAST_PURCHASE",
                col="REGISTRATION_COUNTRY",
                kind="boxen",
                order=["1", "2-5", "6-10", "11-20", "21-50", ">50"],
                col_order=["FIN", "DNK", "GRC"],
                palette=sns.color_palette(['silver']),
                legend=False,
                height=7
                )

g.set_ylabels("DAYS FROM LAST PURCHASE*", fontsize=18)
g.set_xlabels("")
g.set_titles("{col_name}")

# Add horizontal line at 90 days from last purchase
g.axes[0][0].axhline(90, ls='--', color='#00c4e1', linewidth=2)
g.axes[0][1].axhline(90, ls='--', color='#00c4e1', linewidth=2)
g.axes[0][2].axhline(90, ls='--', color='#00c4e1', linewidth=2)


# Customer segmentation

## Finland

### Users by customer segment

In [ ]:
# Data frame with percentage of users
counts = user_data_filtered_df.query("REGISTRATION_COUNTRY == 'FIN'")["PURCHASE_COUNT_SEGM"].value_counts()
percent = user_data_filtered_df.query("REGISTRATION_COUNTRY == 'FIN'")["PURCHASE_COUNT_SEGM"].value_counts(normalize=True)
user_per_df = pd.DataFrame({'Users': counts, 'Percentage': percent}).reset_index()

# Percentage of users
user_per_df.style.format({'Percentage': "{:.2%}"})

### Inactive users by payment method

In [ ]:
# Data frame with percentage of users
counts = user_data_filtered_df.query("REGISTRATION_COUNTRY == 'FIN' and PURCHASE_COUNT_SEGM == 'inactive'")["USER_HAS_VALID_PAYMENT_METHOD"].value_counts()
percent = user_data_filtered_df.query("REGISTRATION_COUNTRY == 'FIN' and PURCHASE_COUNT_SEGM == 'inactive'")["USER_HAS_VALID_PAYMENT_METHOD"].value_counts(normalize=True)
user_per_df = pd.DataFrame({'Users': counts, 'Percentage': percent}).reset_index()

# Percentage of users
user_per_df.style.format({'Percentage': "{:.2%}"})

### One-time shoppers by days from last purchase

In [ ]:
# Data frame with percentage of users
counts = user_data_filtered_df.query("REGISTRATION_COUNTRY == 'FIN' and PURCHASE_COUNT_SEGM == 'one_time_shoppers'")["DAYS_FROM_LAST_PURCHASE_segm"].value_counts()
percent = user_data_filtered_df.query("REGISTRATION_COUNTRY == 'FIN' and PURCHASE_COUNT_SEGM == 'one_time_shoppers'")["DAYS_FROM_LAST_PURCHASE_segm"].value_counts(normalize=True)
user_per_df = pd.DataFrame({'Users': counts, 'Percentage': percent}).reset_index()

# Percentage of users
user_per_df.style.format({'Percentage': "{:.2%}"})

### Casual shoppers by days from last purchase

In [ ]:
# Data frame with percentage of users
counts = user_data_filtered_df.query("REGISTRATION_COUNTRY == 'FIN' and PURCHASE_COUNT_SEGM == 'casual_shoppers'")["DAYS_FROM_LAST_PURCHASE_segm"].value_counts()
percent = user_data_filtered_df.query("REGISTRATION_COUNTRY == 'FIN' and PURCHASE_COUNT_SEGM == 'casual_shoppers'")["DAYS_FROM_LAST_PURCHASE_segm"].value_counts(normalize=True)
user_per_df = pd.DataFrame({'Users': counts, 'Percentage': percent}).reset_index()

# Percentage of users
user_per_df.style.format({'Percentage': "{:.2%}"})

### Frequent shoppers by days from last purchase

In [ ]:
# Data frame with percentage of users
counts = user_data_filtered_df.query("REGISTRATION_COUNTRY == 'FIN' and PURCHASE_COUNT_SEGM == 'frequent_shoppers'")["DAYS_FROM_LAST_PURCHASE_segm"].value_counts()
percent = user_data_filtered_df.query("REGISTRATION_COUNTRY == 'FIN' and PURCHASE_COUNT_SEGM == 'frequent_shoppers'")["DAYS_FROM_LAST_PURCHASE_segm"].value_counts(normalize=True)
user_per_df = pd.DataFrame({'Users': counts, 'Percentage': percent}).reset_index()

# Percentage of users
user_per_df.style.format({'Percentage': "{:.2%}"})

## Denmark

### Users bys customer segment

In [ ]:
# Data frame with percentage of users 
counts = user_data_filtered_df.query("REGISTRATION_COUNTRY == 'DNK'")["PURCHASE_COUNT_SEGM"].value_counts()
percent = user_data_filtered_df.query("REGISTRATION_COUNTRY == 'DNK'")["PURCHASE_COUNT_SEGM"].value_counts(normalize=True)
user_per_df = pd.DataFrame({'Users': counts, 'Percentage': percent}).reset_index()

# Percentage of users
user_per_df.style.format({'Percentage': "{:.2%}"})

### Inactive users by payment method

In [ ]:
# Data frame with percentage of users 
counts = user_data_filtered_df.query("REGISTRATION_COUNTRY == 'DNK' and PURCHASE_COUNT_SEGM == 'inactive'")["USER_HAS_VALID_PAYMENT_METHOD"].value_counts()
percent = user_data_filtered_df.query("REGISTRATION_COUNTRY == 'DNK' and PURCHASE_COUNT_SEGM == 'inactive'")["USER_HAS_VALID_PAYMENT_METHOD"].value_counts(normalize=True)
user_per_df = pd.DataFrame({'Users': counts, 'Percentage': percent}).reset_index()

# Percentage of users 
user_per_df.style.format({'Percentage': "{:.2%}"})

### One-time shoppers by days from last purchase

In [ ]:
# Data frame with percentage of users 
counts = user_data_filtered_df.query("REGISTRATION_COUNTRY == 'DNK' and PURCHASE_COUNT_SEGM == 'one_time_shoppers'")["DAYS_FROM_LAST_PURCHASE_segm"].value_counts()
percent = user_data_filtered_df.query("REGISTRATION_COUNTRY == 'DNK' and PURCHASE_COUNT_SEGM == 'one_time_shoppers'")["DAYS_FROM_LAST_PURCHASE_segm"].value_counts(normalize=True)
user_per_df = pd.DataFrame({'Users': counts, 'Percentage': percent}).reset_index()

# Percentage of users 
user_per_df.style.format({'Percentage': "{:.2%}"})

### Casual shoppers by days from last purchase

In [ ]:
# Data frame with percentage of users 
counts = user_data_filtered_df.query("REGISTRATION_COUNTRY == 'DNK' and PURCHASE_COUNT_SEGM == 'casual_shoppers'")["DAYS_FROM_LAST_PURCHASE_segm"].value_counts()
percent = user_data_filtered_df.query("REGISTRATION_COUNTRY == 'DNK' and PURCHASE_COUNT_SEGM == 'casual_shoppers'")["DAYS_FROM_LAST_PURCHASE_segm"].value_counts(normalize=True)
user_per_df = pd.DataFrame({'Users': counts, 'Percentage': percent}).reset_index()

# Percentage of users 
user_per_df.style.format({'Percentage': "{:.2%}"})

### Frequent shoppers by days from last purchase

In [ ]:
# Data frame with percentage of users 
counts = user_data_filtered_df.query("REGISTRATION_COUNTRY == 'DNK' and PURCHASE_COUNT_SEGM == 'frequent_shoppers'")["DAYS_FROM_LAST_PURCHASE_segm"].value_counts()
percent = user_data_filtered_df.query("REGISTRATION_COUNTRY == 'DNK' and PURCHASE_COUNT_SEGM == 'frequent_shoppers'")["DAYS_FROM_LAST_PURCHASE_segm"].value_counts(normalize=True)
user_per_df = pd.DataFrame({'Users': counts, 'Percentage': percent}).reset_index()

# Percentage of users 
user_per_df.style.format({'Percentage': "{:.2%}"})

## Greece

### Users by customer segment

In [ ]:
# Data frame with percentage of users 
counts = user_data_filtered_df.query("REGISTRATION_COUNTRY == 'GRC'")["PURCHASE_COUNT_SEGM"].value_counts()
percent = user_data_filtered_df.query("REGISTRATION_COUNTRY == 'GRC'")["PURCHASE_COUNT_SEGM"].value_counts(normalize=True)
user_per_df = pd.DataFrame({'Users': counts, 'Percentage': percent}).reset_index()

# Percentage of users per 
user_per_df.style.format({'Percentage': "{:.2%}"})

### Inactive users by payment method

In [ ]:
# Data frame with percentage of users 
counts = user_data_filtered_df.query("REGISTRATION_COUNTRY == 'GRC' and PURCHASE_COUNT_SEGM == 'inactive'")["USER_HAS_VALID_PAYMENT_METHOD"].value_counts()
percent = user_data_filtered_df.query("REGISTRATION_COUNTRY == 'GRC' and PURCHASE_COUNT_SEGM == 'inactive'")["USER_HAS_VALID_PAYMENT_METHOD"].value_counts(normalize=True)
user_per_df = pd.DataFrame({'Users': counts, 'Percentage': percent}).reset_index()

# Percentage of users 
user_per_df.style.format({'Percentage': "{:.2%}"})

### One-time shoppers by days from last purchase

In [ ]:
# Data frame with percentage of users 
counts = user_data_filtered_df.query("REGISTRATION_COUNTRY == 'GRC' and PURCHASE_COUNT_SEGM == 'one_time_shoppers'")["DAYS_FROM_LAST_PURCHASE_segm"].value_counts()
percent = user_data_filtered_df.query("REGISTRATION_COUNTRY == 'GRC' and PURCHASE_COUNT_SEGM == 'one_time_shoppers'")["DAYS_FROM_LAST_PURCHASE_segm"].value_counts(normalize=True)
user_per_df = pd.DataFrame({'Users': counts, 'Percentage': percent}).reset_index()

# Percentage of users 
user_per_df.style.format({'Percentage': "{:.2%}"})

### Casual shoppers by days from last purchase

In [ ]:
# Data frame with percentage of users 
counts = user_data_filtered_df.query("REGISTRATION_COUNTRY == 'GRC' and PURCHASE_COUNT_SEGM == 'casual_shoppers'")["DAYS_FROM_LAST_PURCHASE_segm"].value_counts()
percent = user_data_filtered_df.query("REGISTRATION_COUNTRY == 'GRC' and PURCHASE_COUNT_SEGM == 'casual_shoppers'")["DAYS_FROM_LAST_PURCHASE_segm"].value_counts(normalize=True)
user_per_df = pd.DataFrame({'Users': counts, 'Percentage': percent}).reset_index()

# Percentage of users 
user_per_df.style.format({'Percentage': "{:.2%}"})

### Frequent shoppers by days from last purchase

In [ ]:
# Data frame with percentage of users 
counts = user_data_filtered_df.query("REGISTRATION_COUNTRY == 'GRC' and PURCHASE_COUNT_SEGM == 'frequent_shoppers'")["DAYS_FROM_LAST_PURCHASE_segm"].value_counts()
percent = user_data_filtered_df.query("REGISTRATION_COUNTRY == 'GRC' and PURCHASE_COUNT_SEGM == 'frequent_shoppers'")["DAYS_FROM_LAST_PURCHASE_segm"].value_counts(normalize=True)
user_per_df = pd.DataFrame({'Users': counts, 'Percentage': percent}).reset_index()

# Percentage of users 
user_per_df.style.format({'Percentage': "{:.2%}"})

## Visualize the 4 customer segments

In [ ]:
# Scatter plot for purchase count vs days from last purchase
g = sns.relplot(data=user_data_filtered_df.fillna(0).query("PURCHASE_COUNT <= 100"),
                    x="DAYS_FROM_LAST_PURCHASE",
                    y="PURCHASE_COUNT",
                    hue="PURCHASE_COUNT_SEGM",
                    hue_order=["inactive", "one_time_shoppers",
                               "casual_shoppers", "frequent_shoppers"],
                    col="REGISTRATION_COUNTRY",
                    col_order=["FIN", "DNK", "GRC"],
                    palette=sns.color_palette(
                        ['#980000', '#ff9900', '#00c4e1', '#C0C0C0']),
                    legend=False, 
                    height=6.5
                    )

g.set_xlabels("DAYS FROM LAST PURCHASE", fontsize=18)
g.set_ylabels("PURCHASE COUNT*")
g.set_titles("{col_name}")

# Add horizontal line at 90 days from last purchase
g.axes[0][0].axvline(90, ls='--', color='gray', linewidth=1)
g.axes[0][1].axvline(90, ls='--', color='gray', linewidth=1)
g.axes[0][2].axvline(90, ls='--', color='gray', linewidth=1)

In [ ]:
# Scatter plot for purchase count vs days from last purchase
g = sns.relplot(data=user_data_filtered_df.fillna(0).query("PURCHASE_COUNT <= 100"),
                    x="AVG_PURCHASE_VALUE_EUR",
                    y="PURCHASE_COUNT",
                    hue="PURCHASE_COUNT_SEGM",
                    hue_order=["inactive", "one_time_shoppers",
                               "casual_shoppers", "frequent_shoppers"],
                    col="REGISTRATION_COUNTRY",
                    col_order=["FIN", "DNK", "GRC"],
                    palette=sns.color_palette(
                        ['#980000', '#ff9900', '#00c4e1', '#C0C0C0']),
                    legend=False, 
                    height=6.5
                    )

g.set_xlabels("AVG PURCHASE VALUE €", fontsize=18)
g.set_ylabels("PURCHASE COUNT*")
g.set_titles("{col_name}")

In [ ]:
# Scatter plot for purchase count vs days from last purchase
g = sns.relplot(data=user_data_filtered_df.fillna(0).query("PURCHASE_COUNT <= 100"),
                    x="AVG_DAYS_BETWEEN_PURCHASES",
                    y="PURCHASE_COUNT",
                    hue="PURCHASE_COUNT_SEGM",
                    hue_order=["inactive", "one_time_shoppers",
                               "casual_shoppers", "frequent_shoppers"],
                    col="REGISTRATION_COUNTRY",
                    col_order=["FIN", "DNK", "GRC"],
                    palette=sns.color_palette(
                        ['#980000', '#ff9900', '#00c4e1', '#C0C0C0']),
                    legend=False, 
                    height=6.5
                    )

g.set_xlabels("AVG DAYS BETWEEN PURCHASES", fontsize=18)
g.set_ylabels("PURCHASE COUNT*")
g.set_titles("{col_name}")

## Correlation matrix

In [ ]:
active_users_df.corr()